In [ ]:
!pip install scipy
!pip install accelerate
!pip install bitsandbytes
!pip install transformers
!pip install tokenizers
!pip install SentencePiece
!pip install datasets
!pip install langchain
!pip install sentence-transformers
!pip install chromadb
!pip install ctransformers
!pip install lark

In [3]:
import pandas as pd
from google.colab import drive
import os

In [4]:
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/MyDrive/IR_Project/CTransformer Experimentation")

Mounted at /content/gdrive


In [7]:
# !wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q5_K_M.gguf

--2023-11-19 19:54:31--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q5_K_M.gguf
Resolving huggingface.co (huggingface.co)... 99.84.108.70, 99.84.108.87, 99.84.108.129, ...
Connecting to huggingface.co (huggingface.co)|99.84.108.70|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/46/12/46124cd8d4788fd8e0879883abfc473f247664b987955cc98a08658f7df6b826/c4b062ec7f0f160e848a0e34c4e291b9e39b3fc60df5b201c038e7064dbbdcdc?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.1.Q5_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.1.Q5_K_M.gguf%22%3B&Expires=1700679234&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMDY3OTIzNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80Ni8xMi80NjEyNGNkOGQ0Nzg4ZmQ4ZTA4Nzk4ODNhYmZjNDczZjI0NzY2NGI5ODc5NTVjYzk4YTA4NjU4ZjdkZjZiODI2L2M0YjA2MmVjN2Y

### load the vector store

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

model_kwargs = {"device": "cpu"}  # change to 'cpu' if you don't have a GPU
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs=model_kwargs,
)

vector_store = Chroma(
    "cs_paper_store",
    embeddings_model,
    persist_directory="./chromadb",
)

### load the LLM

In [3]:
from langchain.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

config = {
    'max_new_tokens': 1000,
    'repetition_penalty': 1.1,
    "context_length": 8000,
    "gpu_layers": 0
}

llm = CTransformers(
    model='./mistral-7b-instruct-v0.1.Q5_K_M.gguf',
    model_type="mistral",
    callbacks=[StreamingStdOutCallbackHandler()],
    lib="avx",
    config=config
)

In [4]:
from langchain.prompts import PromptTemplate

rag_template = """You are a helpful bot who reads texts and answers questions about them.
Your expertise lies in the field of academic research and your main priority is ensuring that your answers are correct.
You will be given relevant texts, where each text is enclosed in ``` triple back ticks, and you must derive your answer from the texts as far as possible.
You must to attach a citation for your answers using the texts provided.
If you encounter a question that you do not know how to answer, just mention that you do not know the answer.

Relevant Text:
{context}

Question: {question}

Answer:"""

prompt = PromptTemplate.from_template(rag_template)

In [5]:
import logging
from langchain.retrievers.multi_query import MultiQueryRetriever


logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(), llm=llm,
)

retriever_from_llm.get_relevant_documents("What is a support vector machine?")

INFO:langchain.retrievers.multi_query:Generated queries: ['']


[]

In [6]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="Authors",
        description="The authors of the paper",
        type="string",
    ),
    AttributeInfo(
        name="Published",
        description="The date the paper was published",
        type="string",
    ),
    AttributeInfo(
        name="Summary",
        description="The summary of the paper",
        type="string",
    ),
    AttributeInfo(
        name="Title", description="The title of the paper", type="string"
    ),
    AttributeInfo(
        name="source", description="The URL of the paper", type="string"
    ),
]


document_content_description = "Computer science research paper"


retriever = SelfQueryRetriever.from_llm(
    llm,
    vector_store,
    document_content_description,
    metadata_field_info,
)

In [7]:
logging.getLogger("langchain.retrievers.self_query").setLevel(logging.INFO)

retriever.invoke("What is a support vector machine?")

```json
{
    "query": "support vector machine",
    "filter": "and(eq(\"Title\", \"Support Vector Machine\"), eq(\"Summary\", \"machine learning algorithm that separates data points in a high dimensional space\"))"
}
```

[]